In [ ]:
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import pandas as pd
from sklearn.model_selection import train_test_split

physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices[0])
tf.config.experimental.set_memory_growth(physical_devices[0], True)


In [ ]:
def format_data(data):
    '''

    seperates data first by session, then by lap, before padding each array so that
    they are all the same length for model input.
    Performs test train split also

    '''
    scalers = {}
    sessionUIDs = data.pop('sessionUID')
    for i in data.columns:
        scaler = MinMaxScaler(feature_range=(-1,1))
        s = scaler.fit_transform(data[i].values.reshape(-1,1))
        s = np.reshape(s, len(s))
        scalers['scaler_'+ i ] = scaler
        data[i] = s

    data['sessionUID'] = sessionUIDs
    
    session_groups = data.groupby('sessionUID')
    training_data = []
    target_data = []
    total_laps = 0
    for s in list(session_groups.groups):
        session = session_groups.get_group(s)
        lap_groups = session.groupby('currentLapNum')
        total_laps += len(lap_groups)
        for l in list(lap_groups.groups):
            lap = lap_groups.get_group(l)
            lap = lap.drop('sessionUID', axis=1)
            target_data.append(lap.pop('lap_time_remaining'))
            training_data.append(lap)
    training = [x.to_numpy() for x in training_data]
    target = [y.to_numpy() for y in target_data]
    print(f'Total Laps: {total_laps}')

    max_timesteps = max(training, key=len).shape[0]
    num_rows_to_add = [max_timesteps-l.shape[0] for l in training]

    training_pad = []
    target_pad = []
    print(f'max timesteps : {max_timesteps}')

    for i in range(len(training)):
        rows_to_add = num_rows_to_add[i]

        training_arr = training[i]
        training_append = np.zeros((rows_to_add, training[0].shape[1]), dtype=float)
        training_array = np.vstack((training_arr, training_append))
        training_pad.append(training_array)

        target_arr = target[i]
        target_append = np.zeros((rows_to_add), dtype=float)
        target_array = np.concatenate([target_arr, np.zeros(rows_to_add)])
        target_pad.append(target_array)

    split = int(total_laps*0.9)

    X_train = training_pad[:split]
    X_test = training_pad[split:]
    y_train = target_pad[:split]
    y_test = target_pad[split:]

    X_train = np.array(X_train)
    X_test = np.array(X_test)
    y_test = np.array(y_test)
    y_train = np.array(y_train)

    return X_train, X_test, y_train, y_test, scalers



In [ ]:
import sqlite3
import pandas as pd
from pathlib import Path
import os

def import_training_data():

    dir = '../../SQL_Data/constant_setup'
    files = os.listdir(dir)
    files = [f for f in files if f.endswith('.sqlite3')]

    data = []
    for f in files:
        path = os.path.join(dir, f)
        conn = sqlite3.connect(path)
        if os.path.getsize(path) > 10000:
            cur = conn.cursor()
            cur.execute('SELECT * FROM TrainingData')
            df = pd.DataFrame(cur.fetchall())
            data.append(df)
            
    names = list(map(lambda x: x[0], cur.description))
    df = pd.concat(data)
    df.columns = names
    df = df.drop(['frameIdentifier','bestLapTime', 'pkt_id', 'packetId', 'SessionTime'], axis=1)
    df.set_index('index', inplace=True)

    return df


In [ ]:
# data scaling
data = import_training_data()
data.info()
trainX, testX, trainY, testY, scalers = format_data(data)
print(trainX.shape)

In [ ]:

# create overlapping windows
timesteps = trainX[0].shape[1]-1
look_back = 5
batch_size = 1

Xtrain = trainX[0]
Ytrain = trainY[0]
xtest = testX[0]
ytest = testY[0]

train_generator = tf.keras.preprocessing.sequence.TimeseriesGenerator(Xtrain, Ytrain, length=look_back, sampling_rate=1, stride=1, batch_size=batch_size)
test_generator = tf.keras.preprocessing.sequence.TimeseriesGenerator(xtest, ytest, length=look_back, sampling_rate=1, stride=1, batch_size=1)




In [ ]:
train_gens = []
test_gens = []

for i in range(trainX.shape[0]):
    train_ = tf.keras.preprocessing.sequence.TimeseriesGenerator(trainX[i], trainY[i], length=look_back, sampling_rate=1, stride=1, batch_size=batch_size)
    train_gens.append(train_)


for j in range(testY.shape[0]):
    test_ = tf.keras.preprocessing.sequence.TimeseriesGenerator(testX[j], testY[j], length=look_back, sampling_rate=1, stride=1, batch_size=1)
    test_gens.append(test_)
    


In [ ]:


train_X, train_y = train_generator[0]
test_X, test_y = test_generator[0]


train_samples = train_X.shape[0]*len(train_generator)
test_samples = test_X.shape[0]*len(test_generator)

print("Total Records: {}".format(len(trainX)+len(testX)))
print("Number of samples in training set (.9 * n): trainX = {}".format(trainX.shape[0]))
print("Number of samples in testing set (.1 * n): testX = {}".format(testX.shape[0]))
print("Number of total samples in training feature set: {}".format(train_samples))
print("Number of samples in testing feature set: {}".format(test_samples))


# for i in range(5):
#     x, y = train_generator[i]
#     print(x.shape)

print(trainX.shape)
print(trainY.shape)

In [ ]:
# define model

learning_rate = 0.001
units = 128
epochs = 100
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, LeakyReLU
from tensorflow.keras.callbacks import EarlyStopping

model = Sequential()
model.add(LSTM(units, input_shape=(trainX.shape[1], trainX.shape[2])))
model.add(LeakyReLU(alpha=0.5))
model.add(Dropout(0.1))
model.add(Dense(1))

adam = tf.keras.optimizers.Adam(lr=learning_rate)
callback = [EarlyStopping(monitor="loss", min_delta = 0.0001, patience = 10, mode = 'auto', restore_best_weights=True)] 

model.compile(optimizer=adam, loss='mse', metrics=['mae'])
history = model.fit(train_generator, callbacks=callback, verbose = 1, shuffle=False, epochs=epochs, batch_size=1) 

In [ ]:
pred = model.predict(test_generator)

In [ ]:
from matplotlib import pyplot as plt
y = scalers['scaler_lap_time_remaining'].inverse_transform(pred)
plt.plot(y)
plt.show()